In [1]:
# Библиотеки для обработки табличных данных
import pandas as pd
import numpy as np

# Библиотеки для визуализации графиков и изображений/
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Библиотеки для обучения моделей
import torch
from sklearn.model_selection import train_test_split

# Библиотека для работы с операционной системой
import os
import glob

# Библиотека для парсинга XML-файлов
import xml.etree.ElementTree as ET 

#Вспомогательные библиотеки
from tqdm.auto import tqdm
import shutil as sh
import warnings

# Игнорируем возникающие предупреждения
warnings.filterwarnings('ignore')

c:\Project\lessons\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Далее проверим, что для обучения моделей доступен графический ускоритель. Для этого воспользуемся встроенными возможностями PyTorch:

In [2]:
# Проверим, что нам доступна видеокарта
print(torch.cuda.is_available())
# Узнаем название видеокарты
print(torch.cuda.get_device_name(0))
## True
## Tesla T4

False


AssertionError: Torch not compiled with CUDA enabled

In [3]:
!pwd
## /kaggle/working

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# Путь до папки с датасетом
DATASET_PATH = "/kaggle/input/car-plate-detection"
# Путь до папки с аннотациями
ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "annotations")
# Путь до папки с изображениями
IMAGES_PATH = os.path.join(DATASET_PATH, "images")

print(ANNOTATIONS_PATH)
print(IMAGES_PATH)

## /kaggle/input/car-plate-detection/annotations
## /kaggle/input/car-plate-detection/images

/kaggle/input/car-plate-detection\annotations
/kaggle/input/car-plate-detection\images


In [5]:
# Составляем путь до файла: /путь/до/папки/имя_изображения.png
image_filename = 'Cars129.png'
image_path = os.path.join(IMAGES_PATH, image_filename)
# Читаем изображение
img = plt.imread(image_path)
# Создаём фигуру и координатную плоскость
fig, ax = plt.subplots(figsize=(10, 8))
# Выводим изображение на экран
ax.imshow(img);
# Выводим размер изображения
print('Image shape: {}'.format(img.shape))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/car-plate-detection\\images\\Cars129.png'

In [ ]:
# Инициализируем словарь, на основе которого затем создадим DataFrame
df_dict = {
    "filename":[], "xmin":[], "ymin":[],   
    "xmax":[],"ymax":[], "name":[],    
    "width":[], "height":[],
}

# Создаём цикл по всем аннотациям 
for annotation in glob.glob(ANNOTATIONS_PATH+"/*.xml"):
    # Читаем XML-файл с аннотацией
    tree = ET.parse(annotation)
    # Находим тег, соответствующий имени файла
    filename = tree.find('filename').text
    # Создаём цикл по всем элементам XML-файла
    for elem in tree.iter():
        # Извлекаем из аннотации информацию о размере изображения — ширину и высоту
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    
        # Извлекаем информацию об изображении — имя класса и информацию о координатах bounding box
        if 'object' in elem.tag:
            for attr in list(elem):
                if 'name' in attr.tag:
                    name = attr.text                 
                    df_dict['name'] += [name]
                    df_dict['width'] += [width]
                    df_dict['height'] += [height] 
                    df_dict['filename'] += [filename]


                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            df_dict['xmin'] += [xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            df_dict['ymin'] += [ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            df_dict['xmax'] += [xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            df_dict['ymax'] += [ymax]

In [ ]:
df = pd.DataFrame(df_dict)
df.head()

In [ ]:
print('Data shape: {}'.format(df.shape))
print('Count of unique images: {}'.format(df['filename'].nunique()))
print('Count of classes: {}'.format(df['name'].unique()))
## Data shape: (471, 8)
## Count of unique images: 433
## Count of classes: ['licence']

In [ ]:
# Составляем путь до файла — /путь/до/папки/имя_изображения.png
image_filename = 'Cars129.png'
image_path = os.path.join(IMAGES_PATH, image_filename)
# Читаем изображение
img = plt.imread(image_path)
# Создаём фигуру и координатную плоскость
fig, ax = plt.subplots(figsize=(10, 8))
# Выводим изображение на экран
ax.imshow(img);

# Находим индекс картинки в DataFrame
image_idx = df[df['filename'] == image_filename].index

# Находим координаты левой нижней точки
x_min = df.loc[image_idx, 'xmin'].values
y_min = df.loc[image_idx, 'ymin'].values
# Находим ширину и высоту рамки
box_width = (df.loc[image_idx, 'xmax'] - df.loc[image_idx, 'xmin']).values
box_height = (df.loc[image_idx, 'ymax'] - df.loc[image_idx, 'ymin']).values
print(x_min, y_min, box_width, box_height)

# Так как ограничивающих прямоугольников может быть несколько, создаём цикл по всем.
for i in range(len(image_idx)):
    # Строим прямоугольник
    rect = patches.Rectangle(
        (x_min[i], y_min[i]), #координаты опорной точки
        box_width[i], #ширина прямоугольника
        box_height[i], #высота прямоугольника
        linewidth=3, #ширина линии
        edgecolor='r', #цвет
        facecolor='none' #заливка (none — отсутствует)
    )
    # Накладываем прямоугольник поверх изображения
    ax.add_patch(rect);

In [ ]:
def plot_bounding_box(image_folder, image_filename, df=df):
    # Создаём полный путь до изображения
    image_path = os.path.join(image_folder, image_filename)
    # Читаем изображение
    img = plt.imread(image_path)
    # Создаём фигуру и координатную плоскость
    fig, ax = plt.subplots(figsize=(10, 8))
    # Выводим изображение на экран
    ax.imshow(img);

    # Находим индекс изображения в DataFrame
    image_idx = df[df['filename'] == image_filename].index

    # Находим координаты левой нижней точки
    x_min = df.loc[image_idx, 'xmin'].values
    y_min = df.loc[image_idx, 'ymin'].values
    # Находим ширину и высоту рамки
    box_width = (df.loc[image_idx, 'xmax'] - df.loc[image_idx, 'xmin']).values
    box_height = (df.loc[image_idx, 'ymax'] - df.loc[image_idx, 'ymin']).values
    # Так как ограничивающих прямоугольников может быть несколько, создаём цикл по всем.
    for i in range(len(image_idx)):
        # Строим прямоугольник
        rect = patches.Rectangle(
            (x_min[i], y_min[i]), #координаты опорной точки
            box_width[i], #ширина прямоугольника
            box_height[i], #высота прямоугольника
            linewidth=3, #ширина линии
            edgecolor='r', #цвет
            facecolor='none' #заливка (none — отсутствует)
        )
        # Накладываем прямоугольник поверх изображения
        ax.add_patch(rect);

In [ ]:
plot_bounding_box(image_folder=IMAGES_PATH, image_filename='Cars330.png', df=df)

In [ ]:
count = 3
for i in range(count):
    random_image_idx = np.random.choice(df.shape[0])
    image_filename = df.loc[random_image_idx, 'filename']
    plot_bounding_box(IMAGES_PATH, image_filename, df)

In [ ]:
DATA_PATH = "dataset/" # Корневая папка датасета

LABELS_PATH = os.path.join(DATA_PATH, "labels") # Папка с аннотациями
IMGS_PATH = os.path.join(DATA_PATH, "images") #Папка с изображениями

TRAIN_IMAGES_PATH = os.path.join(IMGS_PATH, "train") #Путь до папки с обучающими изображениями
VAL_IMAGES_PATH = os.path.join(IMGS_PATH, "val") #Путь до папки с валидационными изображениями
TRAIN_LABELS_PATH = os.path.join(LABELS_PATH, "train") #Путь до папки с обучающими аннотациями
VAL_LABELS_PATH = os.path.join(LABELS_PATH, "val") #Путь до папки с валидационными аннотациями

DATA_CONFIG_PATH = os.path.join(DATA_PATH, "dataset.yaml") #Путь до файла конфигураций данных

print(TRAIN_IMAGES_PATH)
print(VAL_IMAGES_PATH)
print(TRAIN_LABELS_PATH)
print(VAL_LABELS_PATH)
print(DATA_CONFIG_PATH)

## dataset/images/train
## dataset/images/val
## dataset/labels/train
## dataset/labels/val
## dataset/dataset.yaml

In [ ]:
# Создаём локальную папку, в которую будем помещать изображения и аннотации к ним (если такая ещё не существует).
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Создаём папку, в которой будем хранить обучающие изображения (если такая ещё не существует).    
if not os.path.exists(TRAIN_IMAGES_PATH):
    os.makedirs(TRAIN_IMAGES_PATH)

# Создаём папку, в которой будем хранить обучающие аннотации к объектам (если такая еще не существует).
if not os.path.exists(TRAIN_LABELS_PATH):
    os.makedirs(TRAIN_LABELS_PATH)

# Создаём папку, в которой будем хранить валидационные изображения (если такая ещё не существует).
if not os.path.exists(VAL_IMAGES_PATH):
    os.makedirs(VAL_IMAGES_PATH)

# Создаём папку, в которой будем хранить валидационные аннотации к объектам (если такая ещё не существует).
if not os.path.exists(VAL_LABELS_PATH):
    os.makedirs(VAL_LABELS_PATH)

In [ ]:
!ls dataset
!ls dataset/images
!ls dataset/labels

## dataset.yaml  images  labels
## train  val
## train  val

In [ ]:
# Создаём содержимое файла
data_config = f'''
path: /kaggle/working/dataset
train: images/train
val: images/val
test:
names:
    0: licence
'''

# Открываем на запись файл, в который записываем конфигурацию путей и из которого будут браться данные для обучения.
with open(DATA_CONFIG_PATH, 'w') as f:
    f.write(data_config)

# Проверяем, что всё записалось корректно.
with open(DATA_CONFIG_PATH, 'r') as f:
    print(f.read())
## path: /kaggle/working/dataset
## train: images/train
## val: images/val
## test:
## names:
##     0: licence